In [493]:
import pandas as pd
import numpy as np

In [494]:
### DEFINE BETA VALUES FOR VDF

beta_vals = {} # TBD: compute values based on growth

beta_template_aux = {
    "Period": ["AM", "MD", "PM", "NT"],
    1: [2.540, 2.543, 2.661, 2.543],
    2: [0.715, 0.708, 0.592, 0.708],
    3: [3.130, 3.133, 3.283, 3.286]
}

beta_vals_aux = pd.DataFrame(beta_template_aux)

beta_vals_aux.set_index("Period", inplace=True)

beta_vals[2025] = beta_vals_aux

In [495]:
### HERE ARE THE MEASURED SPEEDS WE USE AS REFERENCE

measured_speeds_file = r"inputs/measured_speeds.csv"

measured_speeds = pd.read_csv(
    measured_speeds_file,
    sep=",",          # `delimiter` y `sep` son equivalentes; elige uno
    encoding="utf-8",
    decimal=".",      # parsea decimales con punto
    thousands=",",    # parsea separador de miles con coma
    quotechar='"',
    index_col=0
)

measured_speeds

,1NB,1SB,2NB,2SB,3NB,3SB,4NB,4SB,5NB,5SB,6NB,6SB,7NB,7SB,8NB,8SB,9NB,9SB,10NB,10SB
Capacity Factors,,,,,,,,,,,,,,,,,,,,
Night,57.998343,58.906087,62.542391,50.245114,59.918036,43.669434,65.301441,64.301060,68.692699,67.437162,60.602291,66.904842,65.211187,62.941767,65.350604,63.622735,66.774810,63.335906,69.059020,66.184464
AM-Early,57.169795,50.149776,21.039606,66.425744,18.235924,45.327766,60.637052,63.064501,68.692699,67.437162,60.143182,65.710113,64.578069,62.059406,64.204102,63.622735,65.207326,64.211517,66.881483,67.541321
AM-Peak,54.820352,15.592788,11.617289,65.318648,10.686019,18.631080,56.594582,40.991926,65.944991,26.354753,55.907747,28.748174,62.750388,23.589188,63.097135,14.323443,64.005348,17.548992,67.032457,64.598344
AM-Shoulder,50.656781,25.771413,29.080220,63.211595,17.660053,39.092724,57.553812,58.559894,63.408645,61.969284,57.948176,52.195267,64.266097,56.274207,64.963914,60.459174,65.825405,65.725940,69.219997,68.954976
MD,50.656781,28.114269,41.886188,52.961066,24.314565,40.144547,58.546119,63.064501,65.944991,63.690653,58.374265,53.329946,64.266097,59.823031,64.963914,65.527608,66.455313,66.669374,70.201840,69.115710
PM-Shoulder,42.573252,19.429756,29.840487,22.267721,19.283966,26.525137,36.512634,33.462796,25.759762,60.338513,34.072532,64.557304,59.124810,62.644872,62.380122,62.532060,64.301669,55.959435,69.544212,66.931467
PM-Peak,15.632366,15.398687,22.714401,21.652590,15.534306,26.407770,11.998851,35.645153,9.008879,61.969284,18.125343,52.946278,26.713016,60.920701,37.728184,62.532060,51.063090,59.802143,65.843411,67.235011
PM-Late,44.965007,56.228537,62.542391,55.987413,59.918036,44.208562,64.069338,65.587081,65.944991,67.437162,61.068462,66.302095,64.266097,63.544080,64.963914,63.622735,66.138859,64.211517,68.580551,67.235011


In [496]:
measured_speeds.loc['Night']['1NB']

np.float64(57.998343)

In [497]:
### DEFINE LOOKUP TABLE FOR BONUS PER PERIOD

lookup_period_file = r"inputs/LookUp_Period.csv"

lookup_period = pd.read_csv(
    lookup_period_file,
    sep=",",          # `delimiter` y `sep` son equivalentes; elige uno
    encoding="utf-8",
    decimal=".",      # parsea decimales con punto
    thousands=",",    # parsea separador de miles con coma
    quotechar='"',
    index_col=0
)

# Clip y reasignar
lookup_period = lookup_period*0

lookup_period

,Bonus/Mile,4 Periods
Period,,
Night,0.0,
AM-Early,0.0,
AM-Peak,0.0,
AM-Shoulder,0.0,
MD,0.0,
PM-Shoulder,0.0,
PM-Peak,0.0,
PM-Late,0.0,


In [498]:
### DEFINE SEGMENT PARAMETERS
# Default configuration for time periods in traffic data

#TBD: Make this automatically
period_template = [                 # (Period, Hours/Day, Peak/OP, 4Periods tag)
    ("Night",        6, "OP",   "NT"),
    ("AM-Early",     1, "OP",   "AM"),
    ("AM-Peak",      1, "Peak", "AM"),
    ("AM-Shoulder",  3, "OP",   "AM"),
    ("MD",           4, "OP",   "MD"),
    ("PM-Shoulder",  2, "OP",   "PM"),
    ("PM-Peak",      2, "Peak", "PM"),
    ("PM-Late",      5, "OP",   "PM"),
]

rows = []
years = [2025]

# Default time periods list (for reference)
default_time_periods = [
    "Night",
    "AM-Early",
    "AM-Peak",
    "AM-Shoulder",
    "MD",
    "PM-Shoulder",
    "PM-Peak",
    "PM-Late"
]

# Create the base scenario: hour -> time period mapping
hour_to_period = {
    0: "Night",
    1: "Night",
    2: "Night",
    3: "Night",
    4: "Night",
    5: "AM-Early",
    6: "AM-Peak",
    7: "AM-Shoulder",
    8: "AM-Shoulder",
    9: "AM-Shoulder",
    10: "MD",
    11: "MD",
    12: "MD",
    13: "MD",
    14: "PM-Shoulder",
    15: "PM-Shoulder",
    16: "PM-Peak",
    17: "PM-Peak",
    18: "PM-Late",
    19: "PM-Late",
    20: "PM-Late",
    21: "PM-Late",
    22: "PM-Late",
    23: "Night"
}

# Define the segments and their parameters

awt_adt = 1.1 # Average weekday traffic (AWT) to average daily traffic (ADT) ratio
peak_factor = 1.05 # Peak factor for adjustment at peak hour traffic

hov_percentage = pd.DataFrame({
    'Year' : [2025],
    'HOV percentage' : [0]
})

hov_percentage.set_index('Year', inplace=True)



# Define segment parameters base
seg_params = pd.DataFrame({
    'SegDir':   ["1NB","1SB","2NB","2SB","3NB","3SB","4NB","4SB","5NB","5SB","6NB","6SB","7NB","7SB","8NB","8SB","9NB","9SB","10NB","10SB"],
    'Length':    [0.7,0.7,0.7,0.7,0.5,0.5,1.6,1.6,2,2,3.6,3.6,2.9,2.9,3.8,3.8,3.4,3.4,4.5,4.5],
    'Inscope':   [0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8],
    'Lanes_GP':  [4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4], #
    'Lanes_ML':  [2]*20, # Lanes_ML': [2,2,2,2,2,2,2,2,3,3,2,2,2,2], # Do test changing segment 5
    'CapPerLane_GP': [2000]*20,
    'CapPerLane_ML': [1800]*20,
    'Speed_GP':  [55]*6 + [65]*2 + [70]*12,
    'Speed_ML':  [70]*20,
    'Alpha_GP':  [1]*20,
    'Beta_GP':   [6]*20,
    'Alpha_ML':  [1]*20,
    'Beta_ML':   [5]*20,
    'Min_Toll_2016': [None]*20,
    'Max_Toll_2016': [None]*20,
    'LanesGP_AM_Peak': [5]*20,
    'LanesGP_PM_Peak': [5]*20,
})

seg_params.set_index('SegDir', inplace=True)

# Compute capacities as lanes * cap per lane
seg_params['Cap_GP'] = seg_params['Lanes_GP'] * seg_params['CapPerLane_GP']
seg_params['Cap_ML'] = seg_params['Lanes_ML'] * seg_params['CapPerLane_ML']

# Compute peak capacities as Alpha * base capacity
seg_params['CapGP_Peak'] = seg_params['Alpha_GP'] * seg_params['Cap_GP']
seg_params['CapML_Peak'] = seg_params['Alpha_ML'] * seg_params['Cap_ML']

# Optional: if you want integer capacities
seg_params[['Cap_GP','Cap_ML','CapGP_Peak','CapML_Peak']] = seg_params[
    ['Cap_GP','Cap_ML','CapGP_Peak','CapML_Peak']
].astype(int)

# Preview
seg_params

,Length,Inscope,Lanes_GP,Lanes_ML,CapPerLane_GP,CapPerLane_ML,Speed_GP,Speed_ML,Alpha_GP,Beta_GP,Alpha_ML,Beta_ML,Min_Toll_2016,Max_Toll_2016,LanesGP_AM_Peak,LanesGP_PM_Peak,Cap_GP,Cap_ML,CapGP_Peak,CapML_Peak
SegDir,,,,,,,,,,,,,,,,,,,,
1NB,0.7,0.8,4,2,2000,1800,55,70,1,6,1,5,None,None,5,5,8000,3600,8000,3600
1SB,0.7,0.8,4,2,2000,1800,55,70,1,6,1,5,None,None,5,5,8000,3600,8000,3600
2NB,0.7,0.8,4,2,2000,1800,55,70,1,6,1,5,None,None,5,5,8000,3600,8000,3600
2SB,0.7,0.8,4,2,2000,1800,55,70,1,6,1,5,None,None,5,5,8000,3600,8000,3600
3NB,0.5,0.8,4,2,2000,1800,55,70,1,6,1,5,None,None,5,5,8000,3600,8000,3600
3SB,0.5,0.8,4,2,2000,1800,55,70,1,6,1,5,None,None,5,5,8000,3600,8000,3600
4NB,1.6,0.8,4,2,2000,1800,65,70,1,6,1,5,None,None,5,5,8000,3600,8000,3600
4SB,1.6,0.8,4,2,2000,1800,65,70,1,6,1,5,None,None,5,5,8000,3600,8000,3600
5NB,2.0,0.8,4,2,2000,1800,70,70,1,6,1,5,None,None,5,5,8000,3600,8000,3600


In [499]:
capacity_factors_data = {
    "SegDir": ["10NB","10SB","9NB","9SB","8NB","8SB","7NB","7SB","6NB","6SB","5NB","5SB","4NB","4SB","3NB","3SB","2NB","2SB","1NB","1SB"],
    "Night":        [1.00, 1.00, 1.00, 1.00, 1.00, 1.20, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00],
    "AM-Early":     [1.10, 0.75, 1.10, 0.75, 0.75, 1.30, 0.80, 1.00, 1.00, 1.10, 1.35, 1.10, 1.35, 1.10, 1.05, 1.00, 1.05, 1.00, 1.05, 1.00],
    "AM-Peak":      [1.00, 0.75, 1.00, 0.75, 0.70, 1.30, 0.70, 1.00, 0.90, 1.20, 1.10, 1.20, 1.10, 1.20, 0.75, 1.00, 0.75, 1.00, 0.75, 1.00],
    "AM-Shoulder":  [1.20, 0.85, 1.20, 0.85, 1.00, 1.30, 1.00, 1.00, 1.10, 1.20, 1.20, 1.10, 1.20, 1.10, 0.70, 0.90, 0.70, 0.90, 0.70, 0.90],
    "MD":           [1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.10, 1.10, 1.10, 1.00, 1.10, 1.00, 0.85, 0.80, 0.85, 0.80, 0.85, 0.80],
    "PM-Shoulder":  [0.90, 1.00, 0.90, 1.00, 1.20, 1.00, 1.00, 1.00, 1.15, 1.10, 1.10, 0.80, 1.10, 0.80, 0.70, 0.80, 0.70, 0.80, 0.70, 0.80],
    "PM-Peak":      [0.85, 0.90, 0.85, 0.90, 1.00, 1.00, 1.00, 1.00, 1.15, 0.90, 1.00, 0.50, 1.00, 0.50, 0.60, 0.80, 0.60, 0.80, 0.60, 0.80],
    "PM-Late":      [1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00]
}

# Create DF from your original data
df_cf = pd.DataFrame(capacity_factors_data)
# Set 'SegDir' as index
df_cf.set_index('SegDir', inplace=True)

df_cf = df_cf.clip(lower=0.7)

#Set all to 1 for testing
df_cf*=0
df_cf+=1
# Transpose matrix to make the calculation

cap_adj = df_cf.multiply(seg_params['Cap_GP'], axis=0).T

cap_adj

SegDir,10NB,10SB,1NB,1SB,2NB,2SB,3NB,3SB,4NB,4SB,5NB,5SB,6NB,6SB,7NB,7SB,8NB,8SB,9NB,9SB
Night,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0
AM-Early,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0
AM-Peak,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0
AM-Shoulder,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0
MD,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0
PM-Shoulder,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0
PM-Peak,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0
PM-Late,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0,8000.0


In [500]:
import numpy as np

def adjusted_cumprod(row, target_year, multiplier):
    years = row.index
    print(row.values)
    factors = 1 + row.values
    
    # Find the index of the target year
    target_idx = list(years).index(target_year)
    
    # Apply multiplier to the target year's factor
    factors[target_idx] *= multiplier
    
    # Calculate cumulative product
    return pd.Series(np.cumprod(factors), index=years)

In [501]:
### IMPORT GROWTHS FOR EACH CLASS
file_path_growths = r"inputs/growths_per_segment.csv"
base_growth_df = pd.read_csv(
    file_path_growths,
    delimiter=',',
    encoding='utf-8',
    decimal='.',        # ← this tells pandas how to parse decimals
    thousands=',',       # ← this tells pandas how to parse thousands
    quotechar='"'
)

base_growth_df = base_growth_df.iloc[:, 1:]
project_years = base_growth_df.columns[1:].tolist()
base_growth_df.iloc[:, 1:] =  base_growth_df.iloc[:, 1:] + 1

base_growth_df.loc[:, '2032'] *= 1.12

base_growth_df

,SegmentMapped,2026,2027,2028,2029,2030,2031,2032,2033,2034,...,2062,2063,2064,2065,2066,2067,2068,2069,2070,2071
0,S1,1.018928,1.018928,1.018928,1.018928,1.018928,1.022899,1.145647,1.022899,1.022899,...,1.014078,1.014078,1.014078,1.014078,1.008447,1.008447,1.008447,1.008447,1.008447,1.008447
1,S2,1.025583,1.025583,1.025583,1.025583,1.025583,1.031514,1.155296,1.031514,1.031514,...,1.016660,1.016660,1.016660,1.016660,1.009996,1.009996,1.009996,1.009996,1.009996,1.009996
2,S3,1.023025,1.023025,1.023025,1.023025,1.023025,1.028939,1.152412,1.028939,1.028939,...,1.015544,1.015544,1.015544,1.015544,1.009326,1.009326,1.009326,1.009326,1.009326,1.009326
3,S4,1.022150,1.022150,1.022150,1.022150,1.022150,1.027914,1.151263,1.027914,1.027914,...,1.015180,1.015180,1.015180,1.015180,1.009108,1.009108,1.009108,1.009108,1.009108,1.009108
4,S5,1.018077,1.018077,1.018077,1.018077,1.018077,1.022963,1.145719,1.022963,1.022963,...,1.013537,1.013537,1.013537,1.013537,1.008122,1.008122,1.008122,1.008122,1.008122,1.008122
5,S6,1.018314,1.018314,1.018314,1.018314,1.018314,1.023435,1.146247,1.023435,1.023435,...,1.013597,1.013597,1.013597,1.013597,1.008158,1.008158,1.008158,1.008158,1.008158,1.008158
6,S7,1.020046,1.020046,1.020046,1.020046,1.020046,1.025430,1.148481,1.025430,1.025430,...,1.014297,1.014297,1.014297,1.014297,1.008578,1.008578,1.008578,1.008578,1.008578,1.008578
7,S8,1.019537,1.019537,1.019537,1.019537,1.019537,1.025049,1.148055,1.025049,1.025049,...,1.014060,1.014060,1.014060,1.014060,1.008436,1.008436,1.008436,1.008436,1.008436,1.008436
8,S9,1.017924,1.017924,1.017924,1.017924,1.017924,1.022940,1.145693,1.022940,1.022940,...,1.013443,1.013443,1.013443,1.013443,1.008066,1.008066,1.008066,1.008066,1.008066,1.008066
9,S10,1.020342,1.020342,1.020342,1.020342,1.020342,1.024674,1.147635,1.024674,1.024674,...,1.014523,1.014523,1.014523,1.014523,1.008714,1.008714,1.008714,1.008714,1.008714,1.008714


In [502]:
### IMPORT COUNTS AND SEPARATE BY CLASS AND PERIODS

file_path_counts = r"inputs/counts_by_hour_grouped_sorted.csv"
base_counts_df = pd.read_csv(
    file_path_counts,
    delimiter=',',
    encoding='utf-8',
    decimal='.',        # ← this tells pandas how to parse decimals
    thousands=',',       # ← this tells pandas how to parse thousands
    quotechar='"'
)

# --- Ajustar direcciones ---
base_counts_df["Direction"] = base_counts_df["Direction"].replace({"EB": "SB", "WB": "NB"})

# --- Crear columna Seg/Dir ---
base_counts_df["Seg/Dir"] = base_counts_df["Segment"].astype(str) + base_counts_df["Direction"]

# --- Función para procesar cada clase ---
def process_class(df_class):
    # Convertir a formato largo
    df_long = df_class.melt(
        id_vars=["Seg/Dir", "Segment", "Direction", "Class", "AADT"],
        value_vars=[str(h) for h in range(24)],
        var_name="Hour",
        value_name="Volume"
    )
    
    # Mapear hora a periodo
    df_long["Hour"] = df_long["Hour"].astype(int)
    df_long["Period"] = df_long["Hour"].map(hour_to_period)
    
    # Agregar por Segment/Direction/Class/Period
    df_period = df_long.groupby(
        ["Seg/Dir", "Segment", "Direction", "Class", "Period"], as_index=False, sort=False
    ).agg({"Volume": "mean"}).round(0)
    
    # Pivot a formato ancho (periodos como columnas)
    period_order = df_period['Period'].unique()
    df_wide = df_period.pivot(
        index=["Seg/Dir", "Segment", "Direction", "Class"],
        columns="Period",
        values="Volume"
    )[period_order].reset_index()
    
    # Mantener solo Seg/Dir como índice
    df_proc = df_wide.drop(columns=["Class", "Direction", "Segment"]).set_index("Seg/Dir")
    
    return df_proc

# --- Separar por clases y procesar ---
dfs_by_class = {}
for cls in base_counts_df["Class"].unique():
    df_cls = base_counts_df[base_counts_df["Class"] == cls].copy()
    dfs_by_class[cls] = process_class(df_cls)


'''
Vehicle Classifications follow FHWA standards:
Lights: FHWA Classes 1-3 [Light Duty Vehicles]
Medium A: Classes 4-5 [Buses and Single Unit 2 axles trucks] 
Medium B: Class 6-7 [Single Unit 3 or 4 axles Trucks]
Heavy A: Classes 8-10 [Single Trailer 3 or more axles trucks]
Heavy B: Classes 11-13 [Combination Trucks Multitrailer Trucks]
'''

# --- Ejemplo de uso ---
df_lights = dfs_by_class["Lights"]
df_mediumA = dfs_by_class["Medium A"]
df_mediumB = dfs_by_class["Medium B"]
df_heavyA = dfs_by_class["Heavy A"]
df_heavyB = dfs_by_class["Heavy B"]

df_lights

Period,Night,AM-Early,AM-Peak,AM-Shoulder,MD,PM-Shoulder,PM-Peak,PM-Late
Seg/Dir,,,,,,,,
S10NB,692.0,4117.0,5599.0,4097.0,3423.0,4074.0,4314.0,2411.0
S10SB,613.0,1413.0,3167.0,3506.0,3528.0,5372.0,6426.0,2893.0
S1NB,982.0,4676.0,5865.0,4050.0,3666.0,3832.0,3686.0,3364.0
S1SB,1306.0,2435.0,3378.0,3357.0,4074.0,4112.0,3588.0,4084.0
S2NB,886.0,3926.0,5889.0,4718.0,4746.0,4996.0,4726.0,3617.0
S2SB,974.0,2492.0,3551.0,3969.0,5299.0,6208.0,6392.0,4067.0
S3NB,637.0,3032.0,3152.0,2698.0,3070.0,3367.0,3028.0,2334.0
S3SB,1060.0,2991.0,4343.0,4173.0,4694.0,4413.0,3506.0,3693.0
S4NB,786.0,5139.0,6448.0,5014.0,4544.0,5002.0,4945.0,2663.0


In [503]:
# --- Lista de periodos según tus columnas ---
period_cols = ["Night","AM-Early","AM-Peak","AM-Shoulder","MD","PM-Shoulder","PM-Peak","PM-Late"]

# Diccionario de dataframes por clase
class_dfs = {
    "Lights": df_lights,
    "Medium A": df_mediumA,
    "Medium B": df_mediumB,
    "Heavy A": df_heavyA,
    "Heavy B": df_heavyB
}

projected_long_by_class = {}

for cls_name, df_class in class_dfs.items():
    df = df_class.copy()
    
    # Resetear índice Seg/Dir y extraer Segment y Direction
    df = df.reset_index()
    df["Segment"] = df["Seg/Dir"].str.extract(r"(\d+)")[0]    # solo los números
    df["Direction"] = df["Seg/Dir"].str.extract(r"([A-Z]+)")[0]  # solo las letras
    df["Class"] = cls_name
    
    # Melt usando las columnas de periodos
    df_long = df.melt(
        id_vars=["Seg/Dir","Segment","Direction","Class"],
        value_vars=period_cols,
        var_name="Period",
        value_name="AADT "+str(df["Class"][0])
    )
    
    # Normalizar SegDir (opcional)
    df_long["SegDir"] = df_long["Seg/Dir"].str.strip().str.upper().str.lstrip("S")
    
    projected_long_by_class[cls_name] = df_long

# Ejemplo: ver Lights
projected_long_lights_df = projected_long_by_class["Lights"]
projected_long_mediumA_df = projected_long_by_class["Medium A"]
projected_long_mediumB_df = projected_long_by_class["Medium B"]
projected_long_heaviesA_df = projected_long_by_class["Heavy A"]
projected_long_heaviesB_df = projected_long_by_class["Heavy B"]
projected_long_mediumA_df


,Seg/Dir,Segment,Direction,Class,Period,AADT Medium A,SegDir
0,S10NB,10,S,Medium A,Night,32.0,10NB
1,S10SB,10,S,Medium A,Night,26.0,10SB
2,S1NB,1,S,Medium A,Night,34.0,1NB
3,S1SB,1,S,Medium A,Night,32.0,1SB
4,S2NB,2,S,Medium A,Night,23.0,2NB
...,...,...,...,...,...,...,...
155,S7SB,7,S,Medium A,PM-Late,56.0,7SB
156,S8NB,8,S,Medium A,PM-Late,42.0,8NB
157,S8SB,8,S,Medium A,PM-Late,51.0,8SB
158,S9NB,9,S,Medium A,PM-Late,48.0,9NB


In [504]:
rows = []

for year in years:
    for seg in seg_params.index:  # e.g., "1NB", "1SB", etc.
        seg_data = seg_params.loc[seg]
        # Extraer parte numérica y dirección
        seg_numeric = ''.join(filter(str.isdigit, seg))  # e.g., "10"
        direction = seg[len(seg_numeric):]       
        for p, hrs, peak, tag in period_template:
            rows.append({
                "Year": year,
                "SegDir": seg,
                "Segment": seg_numeric,        
                "Direction": direction,     
                "Period": p,
                "Hours/Day": hrs,
                "Peak": peak,
                "4Periods": tag,

                # Parámetros técnicos
                "Length": seg_data["Length"],
                "Speed GP": seg_data["Speed_GP"],
                "Capacity GP": seg_data["CapPerLane_GP"] * seg_data["Lanes_GP"],
                "Alpha GP": seg_data["Alpha_GP"],
                "Beta GP": seg_data["Beta_GP"],
                "Speed ML": seg_data["Speed_ML"],
                "Capacity ML": seg_data["CapPerLane_ML"] * seg_data["Lanes_ML"],
                "Alpha ML": seg_data["Alpha_ML"],
                "Beta ML": seg_data["Beta_ML"],
                "MinToll": 0.5,
                "MinCapture": 0
            })

# --- plantilla base ---
first_model_df = pd.DataFrame(rows)

# --- merge para todas las clases ---
for cls_name, df_proj in projected_long_by_class.items():
    proj_merge_df = df_proj[["SegDir", "Period", f"AADT {cls_name}"]].copy()
    proj_merge_df.rename(columns={"AADT": f"AADT {cls_name}"}, inplace=True)

    first_model_df = first_model_df.merge(
        proj_merge_df,
        on=["SegDir", "Period"],
        how="left"
    )

# --- 1. Reshape growths a formato largo ---
growths_long = base_growth_df.melt(
    id_vars="SegmentMapped",
    var_name="Year",
    value_name="AnnualGrowth"
).copy()
growths_long["Year"] = growths_long["Year"].astype(int)

# --- 2. Calcular crecimiento acumulado desde 2025 ---
# Ordenamos por año y aplicamos cumprod
growths_long = growths_long.sort_values(["SegmentMapped", "Year"])
growths_long["GrowthFactor"] = (growths_long["AnnualGrowth"]).groupby(growths_long["SegmentMapped"]).cumprod()

# Ahora GrowthFactor(y) = factor acumulado 2025→y

# --- 3. Preparar plantilla ---
fm = first_model_df.copy()
fm["Year"] = fm["Year"].astype(int)
fm["Segment"] = fm["Segment"].astype(str).str.replace(r"^S", "", regex=True)
fm["SegmentMapped"] = "S" + fm["Segment"].astype(str)

# --- 4. Merge GrowthFactor ---
fm = fm.merge(
    growths_long[["SegmentMapped", "Year", "GrowthFactor"]],
    on=["SegmentMapped", "Year"],
    how="left"
)

fm["GrowthFactor"] = fm["GrowthFactor"].fillna(1.0)

# --- 5. Aplicar GrowthFactor a todas las clases ---
for cls_name in projected_long_by_class.keys():
    col = f"AADT {cls_name}"
    if col in fm.columns:
        fm[col] = (fm[col].fillna(0) * fm["GrowthFactor"]).round(1)

# --- 6. Limpieza ---
fm = fm.drop(columns=["SegmentMapped"])   # opcional

first_model_df = fm

first_model_df


,Year,SegDir,Segment,Direction,Period,Hours/Day,Peak,4Periods,Length,Speed GP,...,Alpha ML,Beta ML,MinToll,MinCapture,AADT Lights,AADT Medium A,AADT Medium B,AADT Heavy A,AADT Heavy B,GrowthFactor
0,2025,1NB,1,NB,Night,6,OP,NT,0.7,55,...,1,5,0.5,0,982.0,34.0,5.0,254.0,33.0,1.0
1,2025,1NB,1,NB,AM-Early,1,OP,AM,0.7,55,...,1,5,0.5,0,4676.0,70.0,46.0,292.0,20.0,1.0
2,2025,1NB,1,NB,AM-Peak,1,Peak,AM,0.7,55,...,1,5,0.5,0,5865.0,168.0,88.0,286.0,13.0,1.0
3,2025,1NB,1,NB,AM-Shoulder,3,OP,AM,0.7,55,...,1,5,0.5,0,4050.0,156.0,39.0,315.0,7.0,1.0
4,2025,1NB,1,NB,MD,4,OP,MD,0.7,55,...,1,5,0.5,0,3666.0,145.0,41.0,420.0,12.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,2025,10SB,10,SB,AM-Shoulder,3,OP,AM,4.5,70,...,1,5,0.5,0,3506.0,191.0,57.0,417.0,18.0,1.0
156,2025,10SB,10,SB,MD,4,OP,MD,4.5,70,...,1,5,0.5,0,3528.0,152.0,57.0,496.0,16.0,1.0
157,2025,10SB,10,SB,PM-Shoulder,2,OP,PM,4.5,70,...,1,5,0.5,0,5372.0,169.0,56.0,412.0,12.0,1.0
158,2025,10SB,10,SB,PM-Peak,2,Peak,PM,4.5,70,...,1,5,0.5,0,6426.0,156.0,34.0,360.0,8.0,1.0


In [505]:
first_model_df["Capacity GP"] = first_model_df.apply(
    lambda row: cap_adj.loc[row["Period"], row["SegDir"]],
    axis=1
)

first_model_df["B1"] = first_model_df.apply(
    lambda row: beta_vals[row["Year"]].loc[row["4Periods"], 1],
    axis=1
)

first_model_df["B2"] = first_model_df.apply(
    lambda row: beta_vals[row["Year"]].loc[row["4Periods"], 2],
    axis=1
)

# Here we load th value of the counts and we multiply the peak hour values by a constant
lights_w = 1

heavies_w = 3
heavies_w_toll = 3
heavies_w_vot = 3

medium_A_w = 3
medium_A_w_toll = 3
medium_A_w_vot = 3

medium_B_w = 3
medium_B_w_toll = 3
medium_B_w_vot = 3

heavy_A_w = 3
heavy_A_w_toll = 3
heavy_A_w_vot = 3

heavy_B_w = 3
heavy_B_w_toll = 3
heavy_B_w_vot = 3

# 2. Compute TotalLights
first_model_df["TotalLights"] = first_model_df["AADT Lights"] 

first_model_df["TotalMediumA"] = first_model_df["AADT Medium A"]

first_model_df["TotalMediumB"] = first_model_df["AADT Medium B"]

first_model_df["TotalHeavyA"] = first_model_df["AADT Heavy A"]

first_model_df["TotalHeavyB"] = first_model_df["AADT Heavy B"]

first_model_df["TotalVeh"] = first_model_df.apply(
    lambda row: row["TotalLights"] + row["TotalMediumA"] + row["TotalMediumB"] + row["TotalHeavyA"] + row["TotalHeavyB"],
    axis=1
)

first_model_df["Corridor PCE pre-fix"] = first_model_df.apply(
    lambda row: row["TotalLights"] * lights_w + row["TotalMediumA"] * medium_A_w + row["TotalMediumB"] * medium_B_w + row["TotalHeavyA"] * heavy_A_w + row["TotalHeavyB"] * heavy_B_w,
    axis=1
)

first_model_df["Corridor PCE"] = first_model_df["Corridor PCE pre-fix"] # To anulate the suppression


first_model_df.loc[first_model_df['Period'].isin(['AM-Peak', 'PM-Peak']), 'TotalLights'] *= peak_factor

first_model_df["HOV3"] = first_model_df.apply(
    lambda row: row["TotalLights"] * hov_percentage.loc[row['Year']]['HOV percentage'],
    axis=1
)

first_model_df.loc[first_model_df['Period'].isin(['AM-Peak', 'PM-Peak']), 'TotalMediumA'] *= peak_factor
first_model_df.loc[first_model_df['Period'].isin(['AM-Peak', 'PM-Peak']), 'TotalMediumB'] *= peak_factor
first_model_df.loc[first_model_df['Period'].isin(['AM-Peak', 'PM-Peak']), 'TotalHeavyA'] *= peak_factor
first_model_df.loc[first_model_df['Period'].isin(['AM-Peak', 'PM-Peak']), 'TotalHeavyB'] *= peak_factor

first_model_df["TotalVeh"] = first_model_df.apply(
    lambda row: row["TotalLights"] + row["TotalMediumA"] + row["TotalMediumB"] + row["TotalHeavyA"] + row["TotalHeavyB"],
    axis=1
)

first_model_df["InScopeLights"] = first_model_df.apply(
    lambda row: (row["TotalLights"] - row["HOV3"]) * seg_params.loc[row["SegDir"], 'Inscope'],
    axis=1
)

first_model_df["InScopeMediumA"] = first_model_df.apply(
    lambda row: row["TotalMediumA"] * seg_params.loc[row["SegDir"], 'Inscope'],
    axis=1
)

first_model_df["InScopeMediumB"] = first_model_df.apply(
    lambda row: row["TotalMediumB"] * seg_params.loc[row["SegDir"], 'Inscope'],
    axis=1
)

first_model_df["InScopeHeavyA"] = first_model_df.apply(
    lambda row: row["TotalHeavyA"] * seg_params.loc[row["SegDir"], 'Inscope'],
    axis=1
)

first_model_df["InScopeHeavyB"] = first_model_df.apply(
    lambda row: row["TotalHeavyB"] * seg_params.loc[row["SegDir"], 'Inscope'],
    axis=1
)

first_model_df["InScopeVeh"] = first_model_df.apply(
    lambda row: row["TotalVeh"] * seg_params.loc[row["SegDir"], 'Inscope'],
    axis=1
)

# first_model_df.to_csv('model_test.csv')

In [506]:
def get_speed(row):

    max_VC = 1.2  # TBD: check if we need to change this value
    ETC_discount = 0.15


    gp_pce = (
        row["InScopeLights"] * lights_w + 
        row["InScopeMediumA"] * medium_A_w +
        row["InScopeMediumB"] * medium_B_w +
        row["InScopeHeavyA"] * heavy_A_w
    )

    speedGP =  row["Speed GP"] / (1 + row["Alpha GP"] * ((gp_pce / row["Capacity GP"]) ** row["Beta GP"]))

    timeGP = 60 * row["Length"] / speedGP

    gp_vc = gp_pce / row["Capacity GP"]

    return pd.Series([speedGP, timeGP], index=["Speed GP","Time GP"])

In [507]:
first_model_df[["Speed GP Real", "Time GP"]] = first_model_df.apply(
    get_speed, axis=1, result_type='expand'
)

first_model_df.to_csv('model_test.csv')

In [ ]:
def get_capacity(row):

    measured_speed = measured_speeds.loc[row['Period']][row['SegDir']]

    gp_pce = (
        row["TotalLights"] * lights_w + 
        row["TotalMediumA"] * medium_A_w +
        row["TotalMediumB"] * medium_B_w +
        row["TotalHeavyA"] * heavy_A_w
    )

    speed_rate = row['Speed GP'] / measured_speed

    if speed_rate < 1:

        measured_speed = row['Speed GP'] - 0.01

    capacity = gp_pce / ((row['Speed GP'] / (measured_speed * row['Alpha GP']) - 1 / row['Alpha GP']) ** (1 / row['Beta GP']))

    capacity_factor = capacity / row["Capacity GP"]

    return capacity_factor

In [509]:
first_model_df["Capacity Factor"] = first_model_df.apply(
    lambda row: get_capacity(row),
    axis=1
)

first_model_df.to_csv('model_test.csv')

In [510]:
cap_factor = first_model_df.pivot(
    index=["Period"],
    columns="SegDir",
    values="Capacity Factor"
)

cap_factor.to_csv('inputs/capacity_factors.csv')